**Bokeh 심화 -  서울시 코로나 시각화**

---

여태까지 달려오시느라 정말 고생 많으셨습니다!

마지막으로, 저번 파트에서 언급했듯이 보케의 패치 플롯을 사용해 **서울시의 자치구별 코로나 확진자 수**의 추이를 시각화해보도록 하겠습니다.

다음의 내용은 다소 복잡하고 어려울 수 있습니다. 지도를 직접 정의해줘야 하기 때문에 서울시의 자치구별 위/경도 데이터를 직접 수집하고, 알맞은 좌표계로 변환하는 과정이 필요하며, 이와 더불어 코로나 확진자 데이터를 API를 이용해 가져오는 작업이 필요하기 때문입니다. 이 과정에서 데이터에 대한 전처리도 진행되어야 합니다.

아래의 내용들은 코드를 한줄 한줄 해석하시기 보다는, 큰 그림에서 "지금 ~을 하고 있구나" 라고 이해하시고 넘어가시면 좋을 것 같습니다. 

대략적인 흐름을 이해하신 다음, 제 코드를 참고하셔서 여러분들만의 레퍼런스 코드를 만드시고 후에 필요한 상황에서 다시 찾아보실 수 있으시면 좋을 것 같다는 생각이 듭니다.

In [1]:
import pandas as pd
import numpy as np

## 서울 자치구별 데이터 가져오기

---

첫번째 단계로, 서울의 자치구 별 위/경도 좌표를 가져옵니다. 이는 서울시 지도를 직접 만들어주기 위함인데요, 자치구별 경계가 되는 위/경도 값들을 바탕으로 Patch Plot을 그리면 지도가 완성이 될 것 같습니다.

데이터를 수집하는 과정을 일일히 자세하게 설명하지는 않겠습니다. 구체적인 과정이 궁금하신 분들은 저한테 연락을 주시면 상세히 알려드릴 수 있도록 하겠습니다.

데이터 수집에 필요한 모든 파일 및 수집된 데이터는 제 [깃허브](https://github.com/domug/DSL/tree/master/%EC%8B%9C%EA%B0%81%ED%99%94%20%EC%84%B8%EC%85%98) 에서 다운받으실 수 있습니다.

(데이터 수집과정에 관심이 없으신 분들께서는 아래의 내용들을 전부 스킵하셔도 됩니다.)


데이터를 가져오는 대략적인 과정에 대해서 살펴보겠습니다.

---

1. [국가공간정보포탈](http://data.nsdi.go.kr/dataset/15144)에서 서울시의 자치구별 경계에 대한 데이터를 가져옵니다. "LARD_ADM_SECT_SGG_서울" 폴더에 담겨있습니다.

---

2. 일반적으로, 지리 데이터는 .shp의 파일 형식으로 저장되어 있습니다. 이를 파이썬에서 인식할 수 있도록 딕셔너리 형태 (.json)로 변경해주는 작업이 필요합니다. 변경된 json 데이터를 "seoul.json"이라는 이름으로 저장합니다.

---

3. 이렇게 만든 seoul.json 데이터에서 자치구별 지리적 좌표를 얻어올 수 있습니다. 다만 문제점은, 이 좌표계가 우리가 흔히 아는 위도/경도 좌표계가 아니라는 점입니다. 우리나라는 수집된 모든 지리정보에 대해 "UTM-K" 좌표계 (= "GRS80" = "ITRF2000") 를 사용하는데요, 우리에게 익숙한 위/경도 좌표계는 "WGS1984" 좌표계 입니다. 따라서 **좌표계를 변환**해주는 과정이 필요합니다. 이는 추후에 좀 더 자세히 말씀드리겠습니다.

---

4. 변환된 좌표계를 바탕으로, 서울시 자치구별 위/경도 좌표를 가져오면 끝!

In [2]:
# Step 1. 서울시 자치구별 위/경도 데이터 가져오기

import shapefile
import json

# shapefile에서 지리정보 추출하기
file_path = "./LARD_ADM_SECT_SGG_서울//LARD_ADM_SECT_SGG_11.shp"
reader = shapefile.Reader(file_path, encoding='EUC-KR')  # 한국어일때 인코딩 조심
fields = reader.fields[1:]
field_names = [field[0] for field in fields]   # 사용할 필드명
buffer = []

for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))   # 자치구별 지리정보를 필드명과 함께 딕셔너리 형태로 저장
    geom = sr.shape.__geo_interface__         # 자치구별 위/경도
    buffer.append(dict(type="Feature", geometry=geom, properties=atr))  # 지리정보 및 좌표 딕셔너리 형태로 저장
    
    
# 저장할 디렉토리에 GeoJSON 파일 저장
json.dump({"type": "FeatureCollection", "features": buffer}, open("./seoul.json", "w"), ensure_ascii=False)


# 방금 저장한 GeoJson 파일 불러오기
with open('./seoul.json') as json_file:
    dat = json.load(json_file)

In [3]:
# 특정 자치구의 위/경도 예시
dat["features"][0]["geometry"]["coordinates"][0][:3]

[[958208.076291723, 1951847.20746574],
 [958182.846668619, 1951848.52184974],
 [958162.865938656, 1951849.6194628]]

앞서 말한 것처럼 흔히 알고 있는 WGS84 좌표계의 위도/경도가 아니네요. 왜 그런 건지 좀 자세하게 알아봅시다.

앞서 언급한 바와 같이, 일반적으로 우리에게 익숙한 좌표계는 **WGS84** 좌표계입니다. 군사적인 목적으로 만들어져 현재는 전세계적으로 사용되는 일종의 표준 좌표계라고 할 수 있는데요, 지구의 질량 중심을 원점으로 해 가로축을 위도, 세로축을 경도로 지정해 지구를 격자모양으로 나눠 놓은 것이라고 할 수 있습니다. 대표적으로 구글맵이 이 좌표계를 사용합니다.

하지만 여기에서 한가지 문제가 발생하는데요, 지구는 둥글기 때문에 격자모양의 좌표계를 사용할 시 일부 지역에서 **좌표 왜곡**이 발생할 수 밖에 없다는 점입니다. 그렇기 때문에 표준화된 좌표계로는 WGS84가 사용됨에도 불구하고, 일반적으로 좌표 왜곡이 발생하는 지역들은 표준 좌표계에서 약간의 보정치를 추가해 변환한 좌표계를 사용하게 됩니다.

이러한 맥락에서 우리나라는 **UTM-K** 좌표계를 사용하고 있습니다. 현재 데이터의 지리정보도 UTM-K 좌표들인 것 같은데, 후에 구글맵 등을 이용해 지리정보를 시각화 하기 위해서는 WGS84 좌표계로 변환해줄 필요가 있습니다. (물론 이 예시에서는 구글맵을 사용하지 않을 것이지만, 많은 경우 WGS84로 변환한 다음 분석을 진행하는 것이 편합니다.)

따라서 pyproj 모듈을 이용해 우리에게 주어진 좌표들을 WGS84 좌표계로 변환하도록 하겠습니다.

---
좌표계 관련 참고: 

- https://rightstone032.tistory.com/7
- https://jw910911.tistory.com/51

UTM-K 좌표계 변환 공식 참고: 

- https://m.blog.naver.com/PostView.nhn?blogId=hss2864&logNo=221645763282&proxyReferer=https:%2F%2Fwww.google.com%2F

In [4]:
## 좌표계 변환 
from pyproj import Proj, transform
import warnings
warnings.filterwarnings('ignore')

def get_WGS84(coordinates):
    """
    UTM-K 좌표계를 WGS로 변환해주는 함수.
    coordinates: UTM-K 좌표 리스트
    """
    proj_WGS84 = Proj(init="epsg:4326") # WGS1984
    proj_UTMK = Proj('+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 + units=m +no_defs')  # UTM-K 좌표계 변환 공식
    
    long = [x[0] for x in coordinates]  # UTM-K 경도
    lat = [x[1] for x in coordinates]   # UTM-K 위도
    
    coordinates_w84 = transform(proj_UTMK, proj_WGS84, long, lat)
    
    return coordinates_w84

In [5]:
len(dat["features"])

26

In [6]:
dat["features"][0].keys()

dict_keys(['type', 'geometry', 'properties'])

In [7]:
dat["features"][0]["properties"]  # 자치구의 정보

{'ADM_SECT_C': '11200',
 'SGG_NM': '성동구',
 'SGG_OID': 49,
 'COL_ADM_SE': '11200',
 'GID': 260}

In [8]:
dat["features"][0]["geometry"]["coordinates"][0][:3]  # 자치구의 경계 좌표

[[958208.076291723, 1951847.20746574],
 [958182.846668619, 1951848.52184974],
 [958162.865938656, 1951849.6194628]]

In [9]:
dat["features"][0]["properties"]["SGG_NM"]

'성동구'

In [10]:
# 자치구 별 위/경도 변환

dict1 = {}     # 변환된 좌표를 자치구별로 저장해둘 딕셔너리

for i in range(len(dat["features"])):
    name = dat["features"][i]["properties"]["SGG_NM"]                       # 자치구 이름
    coordinates_utmk = dat["features"][i]["geometry"]["coordinates"][0]     # 자치구 좌표
    
    # 좌표 변환
    coordinates_wgs84 = get_WGS84(coordinates_utmk)
    dict1[name] = coordinates_wgs84
    
    print("{} 좌표 변환 완료".format(name))

성동구 좌표 변환 완료
광진구 좌표 변환 완료
강동구 좌표 변환 완료
송파구 좌표 변환 완료
강남구 좌표 변환 완료
서초구 좌표 변환 완료
관악구 좌표 변환 완료
동작구 좌표 변환 완료
영등포구 좌표 변환 완료
금천구 좌표 변환 완료
구로구 좌표 변환 완료


TypeError: must be real number, not list

왜인지 모르게 중간에 오류가 발생했습니다. 11번째 인덱스에서 문제가 발생했는데요, 에러문구를 살펴보니 숫자가 아니라 리스트 자료형이라 변환에 실패했다고 뜹니다. 이를 구체적으로 살펴보도록 합시다.

In [11]:
dat["features"][11]["geometry"]["coordinates"][0][0][:3]

[[935035.24891426, 1950923.29359167],
 [935041.960252126, 1950931.52512409],
 [935055.197629232, 1950937.55279993]]

보니까 다른 자치구와는 다르게 위/경도 좌표들이 세번 충접된 리스트로 저장이 되어 있습니다 (^^..)

따라서 다른 자치구들과는 다르게 한번 더 인덱싱을 해줘야할 것 같네요. 이를 반영해서 위의 코드를 수정하도록 하겠습니다.

In [12]:
# 자치구 별 위/경도 변환

dict1 = {}     # 변환된 좌표를 자치구별로 저장해둘 딕셔너리

for i in range(len(dat["features"])):
    
    name = dat["features"][i]["properties"]["SGG_NM"]                       # 자치구 이름
    
    if i != 11:
        coordinates_utmk = dat["features"][i]["geometry"]["coordinates"][0]        # 자치구 좌표
        
    elif i == 11:
        coordinates_utmk = dat["features"][i]["geometry"]["coordinates"][0][0]     # 자치구 좌표 

    # 좌표 변환
    coordinates_wgs84 = get_WGS84(coordinates_utmk)
    dict1[name] = coordinates_wgs84

    print("{} 좌표 변환 완료".format(name))

성동구 좌표 변환 완료
광진구 좌표 변환 완료
강동구 좌표 변환 완료
송파구 좌표 변환 완료
강남구 좌표 변환 완료
서초구 좌표 변환 완료
관악구 좌표 변환 완료
동작구 좌표 변환 완료
영등포구 좌표 변환 완료
금천구 좌표 변환 완료
구로구 좌표 변환 완료
강서구 좌표 변환 완료
양천구 좌표 변환 완료
마포구 좌표 변환 완료
서대문구 좌표 변환 완료
은평구 좌표 변환 완료
서울시노원구 좌표 변환 완료
서울시도봉구 좌표 변환 완료
강북구 좌표 변환 완료
서울시성북구 좌표 변환 완료
중랑구 좌표 변환 완료
동대문구 좌표 변환 완료
광진구 좌표 변환 완료
용산구 좌표 변환 완료
중구 좌표 변환 완료
종로구 좌표 변환 완료


- 이번엔 제대로 작동하는 것을 확인할 수 있습니다. 
- 근데 현재 자치구의 이름을 보면 "서울시"가 붙어 있는 것도 있고 없는 것도 있네요. 어차피 저희는 서울시의 지도만 사용할 것이기 때문에 편의상 "서울시"를 제거하도록 하겠습니다. 정규표현식을 사용해 이를 수정해주도록 하겠습니다.

In [13]:
# 자치구 별 위/경도 변환
import re

dict1 = {}     # 변환된 좌표를 자치구별로 저장해둘 딕셔너리

for i in range(len(dat["features"])):
    
    name = re.sub("서울시", "", dat["features"][i]["properties"]["SGG_NM"])          # 자치구 이름
    
    if i != 11:
        coordinates_utmk = dat["features"][i]["geometry"]["coordinates"][0]        # 자치구 좌표
        
    elif i == 11:
        coordinates_utmk = dat["features"][i]["geometry"]["coordinates"][0][0]     # 자치구 좌표 

    # 좌표 변환
    coordinates_wgs84 = get_WGS84(coordinates_utmk)
    dict1[name] = coordinates_wgs84

    print("{} 좌표 변환 완료".format(name))

성동구 좌표 변환 완료
광진구 좌표 변환 완료
강동구 좌표 변환 완료
송파구 좌표 변환 완료
강남구 좌표 변환 완료
서초구 좌표 변환 완료
관악구 좌표 변환 완료
동작구 좌표 변환 완료
영등포구 좌표 변환 완료
금천구 좌표 변환 완료
구로구 좌표 변환 완료
강서구 좌표 변환 완료
양천구 좌표 변환 완료
마포구 좌표 변환 완료
서대문구 좌표 변환 완료
은평구 좌표 변환 완료
노원구 좌표 변환 완료
도봉구 좌표 변환 완료
강북구 좌표 변환 완료
성북구 좌표 변환 완료
중랑구 좌표 변환 완료
동대문구 좌표 변환 완료
광진구 좌표 변환 완료
용산구 좌표 변환 완료
중구 좌표 변환 완료
종로구 좌표 변환 완료


- 마지막으로 한가지 이상한 점은 "광진구"가 두번 기록되었다는 점인데요, 아마도 데이터가 중복되어 집계된 것 같습니다.
- 우리는 앞서 자치구별 좌표를 딕셔너리의 형태로 저장해두었는데요, 이 과정에서 광진구의 좌표가 중복되었더라도 결국 마지막 좌표들만 저장되니 크게 지장은 없을 것 같긴합니다. 그래도 혹시 모르니 광진구가 중복되었다는 것을 유념해둡시다.

## 서울시 군별로 시각화

이제 위에서 정리한 위도/경도 데이터를 바탕으로 서울시의 지도를 그려봅시다.

보케의 PatchPlot을 사용하여 진행하도록 하겠습니다.

In [14]:
## 자치구 시각화

from bokeh.models import HoverTool
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource

latitude_list = [dict1[gu][1] for gu in dict1.keys()]    # 모든 자치구의 경도
longitude_list = [dict1[gu][0] for gu in dict1.keys()]   # 모든 자치구의 위도

data = ColumnDataSource(data={
    "Latitude": latitude_list,
    "Longitude": longitude_list,
    "Name": list(dict1.keys())
})


# Create the Hover tool
hover_tool = HoverTool(tooltips=[
    ("Name", "@Name"),
])


# Create figure object
plot = figure(tools=[hover_tool], width=800, height=600)


# Configure the borders of the states
plot.patches("Longitude", "Latitude", source=data, 
             line_color="black", line_width=2, 
             fill_color="gray", fill_alpha=0.1)



# Output the plot
output_notebook()
show(plot)

Loading BokehJS ...

서울시의 지도가 잘 그려졌네요!

Hover Tooltip을 추가했기 때문에 각 자치구의 위치에 마우스 커서를 갖다 대보시면 자치구의 이름을 확인하실 수 있습니다.

이제 이 그래프에 자치구 별 코로나 확진자의 수를 덧입히는 작업을 해보도록 할게요.

## 서울시 코로나 확진자 데이터 가져오기

--- 

서울시 코로나 확진자에 대한 데이터는 [서울 열린데이터 광장](http://data.seoul.go.kr/dataList/OA-20279/S/1/datasetView.do) 에서 가져왔습니다. 

데이터를 크롤링해오기 위해서는 API를 발급받아야 하는데요, 회원가입 하시고 간단하게 신청서 작성하면 바로 발급해주니 발급 받으시면 됩니다.

---

데이터를 수집하는 과정은 함수로 추상화해서 각 함수별로 설명을 달아놨으니 참고하시면 좋겠습니다.

In [15]:
## API 이용해서 서울시 자치구별 코로나 확진자 수 가져오기

import requests as req
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

def get_corona_data(API):
    """
    서울 열린데이터 광장에서 코로나 데이터를 받아와서 확진날짜/거주지역이 담긴 데이터프레임을 반환해주는 함수.
    """
    API_key = API

    dates = []  # 확진 판정 받은 날짜
    area = []   # 확진자가 거주하는 자치구
    
    max_page_number = range(24)  # 21.01.26일 기준입니다. 만약 확진자 수가 더 늘어나면 리스트에 값을 추가하세요.
    # 데이터 가져오기 
    for i in max_page_number:
        URL = "http://openapi.seoul.go.kr:8088/{}/json/Corona19Status/{}000/{}999/".format(API_key, i, i)
        res = req.get(URL)
        soup = res.json()
        for data in soup["Corona19Status"]["row"]:
            dates.append(data["CORONA19_DATE"])
            area.append(data["CORONA19_AREA"])
    
    # 연산 속도 높히기 위해 series로 변환
    #dates = pd.Series(dates)
    #area = pd.Series(area)
    
    # 전처리 후 데이터프레임 생성
    df = pd.DataFrame(data={"확진일": pd.to_datetime(dates), "지역":area})
    #df.날짜 = [pd.to_datetime(x[:-1] + ".20") for x in df.날짜]         # 날짜 데이터 형식 변형
    df = df.sort_values("확진일").reset_index().drop(["index"], axis=1)  # 날짜순으로 정렬
    df["확진자수"] = 1
    return df

In [16]:
API = ""   # 본인의 API
df = get_corona_data(API)
df

,확진일,지역,확진자수
0,2020-01-24,강서구,1
1,2020-01-30,마포구,1
2,2020-01-30,중랑구,1
3,2020-01-30,종로구,1
4,2020-01-31,성북구,1
...,...,...,...
23687,2021-01-26,도봉구,1
23688,2021-01-26,동대문구,1
23689,2021-01-26,용산구,1
23690,2021-01-26,기타,1


- 2020/01/24 ~ 2021/01/26 사이의 코로나 확진자에 대한 데이터가 잘 수집된 것을 확인할 수 있습니다.

In [17]:
print(len(df.지역.unique()))
df.지역.unique()

61


array(['강서구', '마포구', '중랑구', '종로구', '성북구', '타시도', '송파구', '서대문구', '성동구',
       '서초구', '구로구', '강동구 ', '강서구 ', '강동구', '은평구', '노원구 ', '동작구', '노원구',
       '관악구 ', '금천구 ', '강남구', '은평구 ', '양천구', '광진구 ', '동대문구', '성동구 ',
       '관악구', '영등포구', '강남구 ', '성북구 ', '서초구 ', '도봉구 ', '강북구', '용산구',
       '동대문구 ', '중구', '금천구', '구로구 ', '도봉구', '광진구', '기타', '송파구 ', '영등포구 ',
       '중랑구 ', '강북구 ', ' 성동구', '양천구 ', '용산구 ', '서대문구 ', '마포구 ', ' 노원구 ',
       ' 도봉구 ', ' 관악구 ', '타시도 ', '동작구 ', '종로구 ', '중구 ', '', '은평구   ',
       ' 송파구', ' 은평구'], dtype=object)

- 현재 수집된 확진자 데이터를 자세히 살펴봅시다. 지금 집계된 지역이 총 61개인 것을 알 수 있습니다. 상식적으로 서울시에 저렇게 많은 지역구가 있을리가 없죠? 뭔가 문제가 발생한 것 같습니다.

---

- 지역명들을 자세히 살펴보면, 현재 서울시 이외의 지역들이 포함되어 있으며 ("타시도", "기타", "") 여기에 추가적으로 지역구의 명칭에 띄어쓰기가 포함된 것이 있고 아닌 것이 있습니다. 공공기관에서 관리하는 데이터이긴 하지만 아무래도 양식이 개판인 것 같네요..

---
- 이를 적절하게 수정해줘야겠습니다. 우선 정규표현식을 사용해 지역명에서 빈칸을 전부 없애도록 할게요.

In [18]:
import re

# 지역명에 빈칸 삭제
df["지역"] = df["지역"].apply(lambda x: re.sub(r"\s+", "", x))

df["지역"].unique()

array(['강서구', '마포구', '중랑구', '종로구', '성북구', '타시도', '송파구', '서대문구', '성동구',
       '서초구', '구로구', '강동구', '은평구', '노원구', '동작구', '관악구', '금천구', '강남구',
       '양천구', '광진구', '동대문구', '영등포구', '도봉구', '강북구', '용산구', '중구', '기타', ''],
      dtype=object)

- 훨씬 깔끔해졌네요. 다음으로는 서울시의 지역구가 아닌 "타시도", "기타", "" 값들을 제거해주도록 하겠습니다.

- 현재 수집된 확진자 데이터에서 서울시 이외의 지역이 포함된 것을 확인할 수 있는데요 ("타시도", "기타"), 현재 저희는 서울 자치구들의 확진자 수에만 관심이 있으므로 삭제해주도록 하겠습니다.

In [19]:
df = df[df.지역.isin(dict1.keys())]

print(df.shape)
df["지역"].unique()

(20819, 3)


array(['강서구', '마포구', '중랑구', '종로구', '성북구', '송파구', '서대문구', '성동구', '서초구',
       '구로구', '강동구', '은평구', '노원구', '동작구', '관악구', '금천구', '강남구', '양천구',
       '광진구', '동대문구', '영등포구', '도봉구', '강북구', '용산구', '중구'], dtype=object)

- 지역구 명에 대해서 제대로 전처리가 완료된 것 같습니다.

- 다음으로는 누적 확진자 수를 집계해봅시다. 집계에는 판다스의 피벗 테이블 메소드를 사용하겠습니다.

In [20]:
def get_cumsum(pivot_table, date, name):
    """
    특정 요일까지 각 자치구별 누적 환자를 집계하기 위한 함수
    
    pivot_table: 확진일/지역으로 코로나 확진자 수를 집계한 피벗 테이블
    date: 누적 환자 집계 기준 날짜 (~까지)
    name: 집계할 지역구 이름
    """
    
    date = pd.to_datetime(date)                      # 기준이 되는 날짜
    table = pivot_table[pivot_table.index <= date]   # 특정 날짜 이전 까지의 지역구별 확진자 수
    
    cumsum = int(table.loc[:, name].sum())           # 특정 지역구의 누적 확진자 수 
    
    num_new = table.loc[:, name][-1]                 # 신규 확진자 수
    if num_new != num_new:  # 결측치 (NaN) 설정
        num_new = 0
    
    return [cumsum, int(num_new)]

In [21]:
pivot_table = pd.pivot_table(df, index="확진일", columns="지역", aggfunc="sum")["확진자수"]
get_cumsum(pivot_table, "2020.10.15", "강남구")   # "2020.10.15"의 강남구의 누적, 신규 확진자 수

[285, 0]

- 결과가 제대로 출력되는 것 같네요.

## 시각화

---

자 이제 직접 서울시의 지도에 코로나 확진자수 정보를 추가할 차례입니다.

앞서 정의한 함수와 자료들을 종합적으로 사용해서 그래프를 그려보도록 합시다.

그래프에는 자치구별로 **날짜**, **이름**, **위/경도 좌표**, **누적 확진자 수**, **신규 확진자 수** 에 대한 정보가 담길 예정입니다!


In [22]:
## 자치구별 코로나 누적 확진자수 시각화

from bokeh.models import HoverTool, LinearColorMapper, ColorBar
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.palettes import OrRd


# 시각화에 사용될 데이터
pivot_table = pd.pivot_table(df, index="확진일", columns="지역", 
                             aggfunc="sum")["확진자수"]                            # 자치구별 확진자수에 대한 피벗테이블
gu_list = df.지역.unique()                                                        # 자치구 명
latitude_list = [dict1[gu][1] for gu in gu_list]                                 # 모든 자치구의 경도
longitude_list = [dict1[gu][0] for gu in gu_list]                                # 모든 자치구의 위도
start_date = "2020.07.23"                                                        # 집계 기준일
total_list = [get_cumsum(pivot_table, start_date, gu)[0] for gu in gu_list]      # 누적 확진자 수
num_new_list = [get_cumsum(pivot_table, start_date, gu)[1] for gu in gu_list]    # 신규 확진자 수    


# 칼럼 데이터 소스 정의
source = ColumnDataSource(data={
    "latitude": latitude_list,
    "longitude": longitude_list,
    "name": gu_list,
    "date": [start_date for i in range(len(gu_list))],
    "total": total_list,
    "new": num_new_list
})


# 피규어 오브젝트 생성
plot = figure(width=800, height=500, output_backend="webgl")    # WebGL을 사용해서 속도를 높일 수 있음


# 그래프 제목 설정
plot.title.text = "{} 기준 서울시 자치구별 누적확진자 수 분포".format(start_date)
plot.title.align = "center"
plot.title.text_font_size = "25px"


# HoverTool 정의
hover_tool = HoverTool(tooltips=[
    ("Name", "@name"),
    ("Date", "@date"),
    ("Total", "@total"),
    ("New", "@new")
])
plot.add_tools(hover_tool)


# 칼라 팔레트 설정
palette = tuple(reversed(OrRd[9]))
max_total = max([get_cumsum(pivot_table, "2021.01.14", gu)[0] for gu in gu_list])  # 집계 날짜 중 최대 확진자 수

color_mapper = LinearColorMapper(palette=palette, low = 0, high = max_total)


# 패치플롯 생성
plot.patches("longitude", "latitude", source=source, line_color="red", line_width=2,
            fill_color={"field": "total", "transform": color_mapper})


# 칼라바 추가
bar = ColorBar(color_mapper=color_mapper, location=(0,0))
plot.add_layout(bar, "right")


# 그래프 출력
output_notebook()
show(plot)

Loading BokehJS ...

2020/07/23 기준 서울시 자치구별 누적 코로나 확진자 수가 제대로 그려진 것 같습니다!

각 자치구 별로 누적 확진자 수를 나타내기 위해 <code>LinearColorMapper</code> 를 사용했는데요, LinearColorMapper는 칼라맵의 간격을 등차수열로 일정하게 설정합니다. 그래서 현재 그래프에는 최대가 450까지 밖에 없는데요, 만약 값을 늘리고 싶으시다면 <code>LinearColorMapper</code>에서 <code>high</code> 값을 조정하시거나, 값들을 로그변환해준 뒤 기준을 설정하는 <code>LogColorMapper</code> 등을 사용하시면 됩니다.

## "시간" 포함하기

---

대망의 마지막입니다. 

방금 전 그린 그래프에는 **특정 시점** 에서의 누적 확진자 수만 나타나 있는데요, 여기에 "시간"을 고려해서 시간이 지남에 따라 각 자치구의 누적 확진자 수 변화를 시각화할 수는 없을까요?

앞서 플롯에 Interactivity를 더하는 방법을 배웠었죠? 그 내용을 바탕으로 서울에서 코로나 확진자가 발생하기 시작한 20년 1월 24일부터 1주일 단위로 누적 확진자수의 변화를 시각화 해보겠습니다.

여태까지 학습한 모든 내용을 종합적으로 사용해서 시각화를 해야하는만큼 코드가 굉장히 길지만, 한줄씩 차근차근 읽어보시면 크게 어려운 내용은 없을 것(?) 이라고 생각합니다.

In [23]:
import datetime
from bokeh.models import Slider, ColumnDataSource, Label, Button
from bokeh.layouts import layout, widgetbox
from bokeh.plotting import figure, show
import numpy as np



def bkapp(doc):
    
    global df
    global dict1
    
    # 시각화에 사용될 데이터
    pivot_table = pd.pivot_table(df, index="확진일", columns="지역", 
                                 aggfunc="sum")["확진자수"]                            # 자치구별 확진자수에 대한 피벗테이블
    gu_list = df.지역.unique()                                                        # 자치구 명
    latitude_list = [dict1[gu][1] for gu in gu_list]                                 # 모든 자치구의 경도
    longitude_list = [dict1[gu][0] for gu in gu_list]                                # 모든 자치구의 위도
    start_date = "{}.{}.{}".format(min(pivot_table.index).year, 
                                   min(pivot_table.index).month, 
                                   min(pivot_table.index).day)                       # 시작일
    total_list = [get_cumsum(pivot_table, start_date, gu)[0] for gu in gu_list]      # 누적 확진자 수
    num_new_list = [get_cumsum(pivot_table, start_date, gu)[1] for gu in gu_list]    # 신규 확진자 수    
    
    
    # 칼럼 데이터 소스 정의
    source = ColumnDataSource(data={
        "latitude": latitude_list,
        "longitude": longitude_list,
        "name": gu_list,
        "date": [start_date for i in range(len(gu_list))],
        "total": total_list,
        "new": num_new_list
    })


    # 피규어 오브젝트 생성
    plot = figure(width=800, height=500, output_backend="webgl")    # WebGL을 사용해서 속도를 높일 수 있음


    # 그래프 제목 설정
    plot.title.text = "{} 기준 서울시 자치구별 누적확진자 수 분포".format(start_date)
    plot.title.align = "center"
    plot.title.text_font_size = "25px"
    
    
    # HoverTool 정의
    hover_tool = HoverTool(tooltips=[
        ("Name", "@name"),
        ("Date", "@date"),
        ("Total", "@total"),
        ("New", "@new")
    ])
    plot.add_tools(hover_tool)
    
    
    # 칼라 팔레트
    palette = tuple(reversed(OrRd[9]))
    max_total = max([get_cumsum(pivot_table, "2021.01.14", gu)[0] for gu in gu_list])  # 집계 날짜 중 최대 확진자 수
    
    color_mapper = LinearColorMapper(palette=palette, low = 0, high = max_total)

    
    # 패치플롯 생성
    plot.patches("longitude", "latitude", source=source, line_color="red", line_width=2,
                fill_color={"field": "total", "transform": color_mapper})


    # 칼라바 추가
    bar = ColorBar(color_mapper=color_mapper, location=(0,0))
    plot.add_layout(bar, "right")

    
    ############################ 여기까지는 동일 ##############################
    
    
    # Define the callback function
    min_date = min(pivot_table.index)      # 집계 시작 날짜
    max_date = max(pivot_table.index)      # 집계 최대 날짜
    
    def slider_update(attr, old, new):
        select = int(slider.value)
        new_date = min_date + datetime.timedelta(days=select)
        new_date = "{}.{}.{}".format(new_date.year, new_date.month, new_date.day)
        
        # 새롭게 갱신되어야 할 값들 집계
        total_new = [get_cumsum(pivot_table, new_date, gu)[0] for gu in gu_list]       # 누적 확진자 수
        num_new = [get_cumsum(pivot_table, new_date, gu)[1] for gu in gu_list]         # 신규 확진자 수
        
        # 칼럼 데이터 소스 재정의
        source.data = {
            "latitude": latitude_list,
            "longitude": longitude_list,
            "name": gu_list,
            "date": [new_date for i in range(len(gu_list))],
            "total": total_new,
            "new": num_new
        }
        
        # 그래프 제목 갱신
        plot.title.text = "{} 기준 서울시 자치구별 누적확진자 수 분포".format(new_date)

        
    def animate_update():
        if slider.value < (max_date - min_date).days:
            slider.value += 1
        else:
            slider.value = 0
    
    
    # Create Slider Widget
    slider = Slider(start=0, end = (max_date - min_date).days, 
                    step=1, value=0, title="Days")
    
    slider.on_change("value", slider_update)
    
    
    # 애니메이션
    callback_id = None
    def animate():
        global callback_id
        if button.label == '► Play':
            button.label = '❚❚ Pause'
            callback_id = doc.add_periodic_callback(animate_update, 100)
        else:
            button.label = '► Play'
            doc.remove_periodic_callback(callback_id)
            
    button = Button(label='► Play', width=60)
    button.on_click(animate)
            
    
    # Application on Jupyter notebook
    doc.add_root(layout([plot], [slider, button], sizing_mode='stretch_width'))

In [24]:
show(bkapp)

위 그래프에서 슬라이더 위젯을 변경함에 따라 각 자치구별 색상이 변하는 것을 볼 수 있습니다.

확실히 연말부터 확진자가 폭등한 것을 시각적으로 확인할 수 있네요. 하루 빨리 코로나가 진정되어 오프라인에서 학회원 분들을 뵐 수 있었으면 좋겠습니다 ㅠㅠ

이 예시에서는 "누적 확진자 수"와 "신규 확진자 수" 두 개의 정보만을 이용해서 시각화를 진행했는데요, 추가적으로 더 많은 정보를 담고 싶으시다면 칼럼 데이터 소스에 데이터를 추가하시면 훨씬 더 의미 있는 그래프를 그리실 수 있으실 거라고 생각합니다.

시각화는 여러분들께서 분석하신 인사이트를 남들에게 전달할 수 있는 강력한 도구입니다. 보케가 여러분들의 경쟁력이 될 수 있기를 희망합니다.